In [315]:
import random
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report

In [316]:
def get_npi_ccspcs_dict():
    npi_ccspcs_features = pd.read_pickle('./ccspcs_npi_features.pkl')
    npi_ccspcs_features_np = np.array(npi_ccspcs_features)
    npi_ccspcs_features_dict = {}
    for row in npi_ccspcs_features_np:
        if sum(row[1:]) > 0:
            npi_ccspcs_features_dict[row[0]] = row[1:]
    return npi_ccspcs_features_dict

In [317]:
def get_npi_features(npi_upper_level_features_path = "./npi_upper_features.pkl"):
    npi_ccspcs_features_dict = get_npi_ccspcs_dict()
    npi_upper_level_features = pd.read_pickle(npi_upper_level_features_path)
    # npi_cpt_features = pd.read_pickle('./chuncked_npi_ncpcs_2019_0_.pkl')
    return npi_ccspcs_features_dict, npi_upper_level_features #, npi_cpt_features

In [318]:
def get_indigo_spl_npi(npi_ccspcs_features_dict, npi_indigo_spl):
    indigo_spl_npi = {}
    for npi in npi_ccspcs_features_dict:
        if int(npi) in npi_indigo_spl:
            if npi_indigo_spl[int(npi)] not in indigo_spl_npi:
                indigo_spl_npi[npi_indigo_spl[int(npi)]] = []
            indigo_spl_npi[npi_indigo_spl[int(npi)]].append(npi)
    return indigo_spl_npi

In [319]:
def get_spl_specific_indigo_spl_npi(npi_upper_level_features, npi_ccspcs_features_dict, npi_indigo_spl_path = './npi_indigo_spl.pkl'):
    npi_indigo_spl = pd.read_pickle(npi_indigo_spl_path)
    indigo_spl_npi_upper = get_indigo_spl_npi(npi_upper_level_features, npi_indigo_spl)
    indigo_spl_npi_ccspcs = get_indigo_spl_npi(npi_ccspcs_features_dict, npi_indigo_spl)
    # indigo_spl_npi_cpt_features = get_indigo_spl_npi(npi_cpt_features, npi_indigo_spl)
    return indigo_spl_npi_upper, indigo_spl_npi_ccspcs

In [320]:
def get_X_y_v2(indigo_spl_npi, outliers_path, all_specilities, npi_features):
    X = []; y = []; speciality_label = {}; label = 0
    for spl in all_specilities:
        for granular_spl in all_specilities[spl]:
            speciality_label[granular_spl] = label
            outlier_cur = pd.read_pickle(outliers_path + "outlier_npis" + granular_spl + ".pkl")
            for npi in indigo_spl_npi[granular_spl]:
                if npi not in outlier_cur:
                    X.append(npi_features[npi])
                    y.append(label)
            label += 1
    return X, y, speciality_label

In [321]:
def alter_y(speciality_lis, X, y, speciality_label, spl_exclude=None):
    altered_y = []
    label_lis = [speciality_label[spl] for spl in speciality_label if spl in speciality_lis]
    if spl_exclude is None:
        for label in y:
            if label in label_lis:
                altered_y.append(1)
            else:
                altered_y.append(0)
        return X, altered_y
    else:
        exclude_label_lis = [speciality_label[spl] for spl in speciality_label if spl in spl_exclude]
        altered_X = []
        i = 0
        for label in y:
            if label in label_lis:
                altered_y.append(1)
                altered_X.append(X[i])
            elif label not in exclude_label_lis:
                altered_y.append(0)
                altered_X.append(X[i])                
            i += 1
        return altered_X, altered_y

In [322]:
def get_results(y_train, y_test, X_train, X_test, random_search, spl):
    print("X_train results,")
    predicted_probabilities = random_search.best_estimator_.predict_proba(X_train)
    predictions = np.argmax(predicted_probabilities, axis=1)
    #print(classification_report(y_train, predictions, target_names=['Others', spl]))
    train_out = classification_report(y_train, predictions, target_names=['Others', spl], output_dict = True)

    print("X_test results,")
    predicted_probabilities = random_search.best_estimator_.predict_proba(X_test)
    predictions = np.argmax(predicted_probabilities, axis=1)
    #print(classification_report(y_test, predictions, target_names=['Others', spl]))
    test_out = classification_report(y_test, predictions, target_names=['Others', spl], output_dict = True)

    # number of spl predicted as others
    count_spl_others = 0
    for i in range(len(y_test)):
        if y_test[i] == 1 and predictions[i] == 0:
            count_spl_others += 1
    print("number of spl predicted as others: ", count_spl_others)

    # num of others predicted as spl
    count_others_spl = 0
    for i in range(len(y_test)):
        if y_test[i] == 0 and predictions[i] == 1:
            count_others_spl += 1
    print("num of others predicted as spl: ", count_others_spl)

    # num of spl predicted as spl
    count_spl_spl = 0
    for i in range(len(y_test)):
        if y_test[i] == 1 and predictions[i] == 1:
            count_spl_spl += 1
    print("num of spl predicted as spl: ", count_spl_spl)

    # num of others predicted as others
    count_others_others = 0
    for i in range(len(y_test)):
        if y_test[i] == 0 and predictions[i] == 0:
            count_others_others += 1
    print("num of others predicted as spl: ", count_others_others)

    return train_out, test_out, count_spl_others, count_others_spl

In [323]:
def add_results(results_train, results_test, spl, train_out, test_out):
    results_train.append(["CPT upper level features", "Train", spl, train_out['Others']['precision'], train_out['Others']['recall'], train_out[spl]['precision'], train_out[spl]['recall'], train_out['macro avg']['f1-score']])
    results_test.append(["CPT upper level features", "Test", spl, test_out['Others']['precision'], test_out['Others']['recall'], test_out[spl]['precision'], test_out[spl]['recall'], test_out['macro avg']['f1-score']])
    return results_train, results_test

In [324]:
def train_model(X, original_y, param_dist, spl, npi_speciality_label, concerned_specialities, results_train, results_test):
    X_train, X_test, y_train, y_test = train_test_split(X, original_y, test_size=0.3, random_state=798, stratify=original_y)
    X_train, y_train = alter_y(concerned_specialities[spl], X_train, y_train, npi_speciality_label)
    X_test, altered_y_test = alter_y(concerned_specialities[spl], X_test, y_test, npi_speciality_label)
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=798, stratify=y)
    model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(set(y_train)), tree_method='gpu_hist', gpu_id=0)
    random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, scoring='accuracy', n_jobs=-1, cv=2, verbose=3, random_state=53)
    random_search.fit(X_train, y_train)
    print("Best parameters found: ", random_search.best_params_)

    train_out, test_out, count_spl_others, count_others_spl = get_results(y_train, altered_y_test, X_train, X_test, random_search, spl)
    results_train, results_test = add_results(results_train, results_test, spl, train_out, test_out)
    for granular_spl in concerned_specialities[spl]:
        print(list(set(concerned_specialities[spl]) - set([granular_spl])))
        altered_X_test, altered_y_test = alter_y([granular_spl], X_test, y_test, npi_speciality_label, list(set(concerned_specialities[spl]) - set([granular_spl])))
        train_out, test_out, count_spl_others, count_others_spl = get_results(y_train, altered_y_test, X_train, altered_X_test, random_search, granular_spl)
        results_train, results_test = add_results(results_train, results_test, granular_spl, train_out, test_out)


    #return train_out, test_out, count_spl_others, count_others_spl, random_search
    return results_train, results_test

In [325]:
h = [1, 2, 3]
z = [2]
list(set(h) - set(z))

[1, 3]

In [326]:
def train_binary_spl_predict_mdoel(concerned_specialities, all_specilities, param_dist):
    
    npi_ccspcs_features_dict, npi_upper_level_features = get_npi_features()
    indigo_spl_npi_upper, indigo_spl_npi_ccspcs = get_spl_specific_indigo_spl_npi(npi_upper_level_features, npi_ccspcs_features_dict)

    results_test = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1"]]
    results_train = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1"]]

    npi_upper_X, npi_upper_y, npi_upper_speciality_label = get_X_y_v2(indigo_spl_npi_upper, "./outliers_v2/", all_specilities, npi_upper_level_features)
    #npi_ccspcs_X, npi_ccspcs_y, npi_ccspcs_speciality_label = get_X_y_v2(indigo_spl_npi_ccspcs, "./outliersccspcs/", all_specilities, npi_ccspcs_features_dict)
    print('Starting the run!!')
    


    for spl in concerned_specialities:
        # try:
        #altered_y = alter_y(concerned_specialities[spl], npi_upper_y, npi_upper_speciality_label)
        results_train, results_test = train_model(npi_upper_X, npi_upper_y, param_dist, spl, npi_upper_speciality_label, concerned_specialities, results_train, results_test)
        #results_train, results_test = train_model(npi_ccspcs_X, npi_ccspcs_y, param_dist, spl, npi_ccspcs_speciality_label, concerned_specialities, results_train, results_test)

        # results_train.append(["CPT upper level features", "Train", spl, train_out['Others']['precision'], train_out['Others']['recall'], train_out[spl]['precision'], train_out[spl]['recall'], train_out['macro avg']['f1-score']])
        # results_test.append(["CPT upper level features", "Test", spl, test_out['Others']['precision'], test_out['Others']['recall'], test_out[spl]['precision'], test_out[spl]['recall'], test_out['macro avg']['f1-score']])
        # except:
        #     print('\033[1m' + '\033[91m' + spl + " not in CPT upper level features" + '\033[0m' + '\033[0m')

        # try:
        # altered_y = alter_y(spl, npi_ccspcs_y, npi_ccspcs_speciality_label)
        # train_out, test_out, count_spl_others, count_others_spl = train_model(npi_ccspcs_X, altered_y, param_dist, spl)
        # results_train.append(["CCSPCS level 3 features", "Train", spl, train_out['Others']['precision'], train_out['Others']['recall'], train_out[spl]['precision'], train_out[spl]['recall'], train_out['macro avg']['f1-score']])
        # results_test.append(["CCSPCS level 3 features", "Test", spl, test_out['Others']['precision'], test_out['Others']['recall'], test_out[spl]['precision'], test_out[spl]['recall'], test_out['macro avg']['f1-score']])
        # except:
        #     print('\033[1m' + '\033[91m' + spl + " not in CCSPCT level 3 features" + '\033[0m' + '\033[0m')

        results_train.append(len(results_train[0])*[""])
        results_test.append(len(results_test[0])*[""])

        print('\033[1m' + '\033[92m' + "Done: " + spl + '\033[0m' + '\033[0m')
    
    df = pd.DataFrame(results_test + results_train)
    #print(df)
    df.to_csv('./binary_spl_prediction_granular_output_v3.csv', index=False)

In [327]:
all_specialities = {
    'Nurse': ['Advanced Practice Registered Nurse (APRN)',
    'Certified Nurse Practitioner',
    'Clinical Nurse Specialist',
    'Nurse - Student',
    'Certified Nurse Midwife (CNM)',
    'Nurse'],
    'Anesthesiology': ['Anesthesiology', 'Anesthesiology Assistant (AA)', 'CRNA'],
    'Cardiovascular': ['Cardiovascular Disease-Minor Surgery',
    'Cardiovascular Disease-No Surgery',
    'Cardiovascular Disease-Surgery',
    # 'Cardiac Technician',
    'Vascular-Surgery'],
    'Dermatology': ['Dermatology-Minor Surgery', 'Dermatology-No Surgery'],
    'Neurology': ['Neurology-Surgery', 'Neurology-No Surgery'],
    'Gynecology': ['Obstetrics Gynecology-Surgery',
    'Gynecology-Minor Surgery',
    'Gynecology-No Surgery',
    'Gynecology-Surgery'],
    'General': ['General Preventive Med-No Surgery', 'General NOC-Surgery'],
    'Orthopedic': ['Orthopedic Excl Back-Surgery',
    'Orthopedic Incl Back-Surgery'],
    'Ophthalmology': ['Ophthalmology-No Surgery',
    'Ophthalmology-Minor Surgery',
    'Ophthalmology-Surgery'],
    'Otorhinolaryngology': ['Otorhinolaryngology-No Surgery',
    'Otorhinolaryngology-Minor Surgery',
    'Otorhinolaryngology-Surgery'],
    'Radiology': ['Radiology Diagnostic-Minor Surgery',
    'Radiology Diagnostic-No Surgery'],
    'Pediatrics': ['Pediatrics-Minor Surgery', 'Pediatrics-No Surgery'],
    'Acupuncture': ['Acupuncture'],
    'Aerospace Medicine': ['Aerospace Medicine'],
    'Allergy': ['Allergy'],
    'Audiologist': ['Audiologist'],
    'Bariatric-Surgery': ['Bariatric-Surgery'],
    'Chiropractor': ['Chiropractor'],
    'Colon And Rectal-Surgery': ['Colon And Rectal-Surgery'],
    'Counselor': ['Counselor'],
    'Dietitian': ['Dietitian'],
    'Emergency Med-No Surgery': ['Emergency Med-No Surgery'],
    'Emergency Medical Technician (EMT)': ['Emergency Medical Technician (EMT)'],
    'Endocrinology-No Surgery': ['Endocrinology-No Surgery'],
    'Family Medicine-No Surgery': ['Family Medicine-No Surgery'],
    'Forensic Medicine': ['Forensic Medicine'],
    'Gastroenterology-No Surgery': ['Gastroenterology-No Surgery'],
    'Geriatrics-No Surgery': ['Geriatrics-No Surgery'],
    'Hand-Surgery': ['Hand-Surgery'],
    'Hematology-No Surgery': ['Hematology-No Surgery'],
    'Hospitalists': ['Hospitalists'],
    'Infectious Diseases-No Surgery': ['Infectious Diseases-No Surgery'],
    'Internal Medicine-No Surgery': ['Internal Medicine-No Surgery'],
    'Medical Assistant': ['Medical Assistant'],
    'Neonatology': ['Neonatology'],
    'Nephrology-No Surgery': ['Nephrology-No Surgery'],
    'Nuclear Medicine': ['Nuclear Medicine'],
    'O.R. Technician': ['O.R. Technician'],
    'Occupation Therapist': ['Occupation Therapist'],
    'Occupational Medicine': ['Occupational Medicine'],
    'Optometrist': ['Optometrist'],
    'Pain Medicine': ['Pain Medicine'],
    'Pathology-No Surgery': ['Pathology-No Surgery'],
    'Pharmacist': ['Pharmacist'],
    'Phlebology': ['Phlebology'],
    'Physiatry': ['Physiatry'],
    'Physical Therapist': ['Physical Therapist'],
    'Physician Assistant': ['Physician Assistant'],
    'Physicians NOC-No Surgery': ['Physicians NOC-No Surgery'],
    'Physiotherapist': ['Physiotherapist'],
    'Plastic Surgery': ['Plastic NOC-Surgery', 'Plastic Otorhinolaryngology-Surgery'],
    'Podiatrist': ['Podiatrist'],
    'Psychiatry': ['Psychiatry'],
    'Psychologist': ['Psychologist'],
    'Pulmonary Diseases-No Surgery': ['Pulmonary Diseases-No Surgery'],
    'Radiation Therapy NOC': ['Radiation Therapy NOC'],
    'Respiratory Therapist': ['Respiratory Therapist'],
    'Rheumatology-No Surgery': ['Rheumatology-No Surgery'],
    'Social Worker': ['Social Worker'],
    'Sonographer': ['Sonographer'],
    'Surgeon Assistant': ['Surgeon Assistant'],
    'Thoracic-Surgery': ['Thoracic-Surgery'],
    'Traumatic-Surgery': ['Traumatic-Surgery'],
    'Urgent Care-No Surgery': ['Urgent Care-No Surgery'],
    'Urological-Surgery': ['Urological-Surgery'],
    # 'Pharmacology Clinica': ['Pharmacology Clinica']
 }

In [328]:
concerned_specialities = {
    'Anesthesiology Group': ['Anesthesiology', 'Anesthesiology Assistant (AA)', 'CRNA'],
}

In [329]:
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [6, 7, 8, 9, 10, 11, 12],
    'colsample_bytree': [0.6, 0.65, 0.7, 0.75],
}

In [330]:
train_binary_spl_predict_mdoel(all_specialities, all_specialities, param_dist)

Starting the run!!
Fitting 2 folds for each of 10 candidates, totalling 20 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best parameters found:  {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
X_train results,
X_test results,
number of spl predicted as others:  5863
num of others predicted as spl:  127
num of spl predicted as spl:  395
num of others predicted as spl:  30565
['Nurse', 'Clinical Nurse Specialist', 'Nurse - Student', 'Certified Nurse Practitioner', 'Certified Nurse Midwife (CNM)']
X_train results,
X_test results,
number of spl predicted as others:  568
num of others predicted as spl:  127
num of spl predicted as spl:  26
num of others predicted as spl:  30565
['Advanced Practice Registered Nurse (APRN)', 'Nurse', 'Clinical Nurse Specialist', 'Nurse - Student', 'Certified Nurse Midwife (CNM)']
X_train results,
X_test results,
number of spl predicted as others:  4161
num of others predicted as spl:  127
num of spl predicted as spl:  360
num of others predicted as spl:  30565
['Advanced Practice Registered Nurse (APRN)', 'Nurse', 'Nurse - Student', 'Certifi

c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best parameters found:  {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
X_train results,
X_test results,
number of spl predicted as others:  96
num of others predicted as spl:  203
num of spl predicted as spl:  1960
num of others predicted as spl:  34691
['Anesthesiology Assistant (AA)', 'CRNA']
X_train results,
X_test results,
number of spl predicted as others:  71
num of others predicted as spl:  203
num of spl predicted as spl:  920
num of others predicted as spl:  34691
['Anesthesiology', 'CRNA']
X_train results,
X_test results,
number of spl predicted as others:  1
num of others predicted as spl:  203
num of spl predicted as spl:  54
num of others predicted as spl:  34691
['Anesthesiology', 'Anesthesiology Assistant (AA)']
X_train results,
X_test results,
number of spl predicted as others:  24
num of others predicted as spl:  203
num of spl predicted as spl:  986
num of others predicted as spl:  34691
Done: Anesthesiology
Fitting 2 folds for ea

...

Best parameters found:  {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
X_train results,
X_test results,
number of spl predicted as others:  33
num of others predicted as spl:  1
num of spl predicted as spl:  0
num of others predicted as spl:  36916
[]
X_train results,
X_test results,
number of spl predicted as others:  33
num of others predicted as spl:  1
num of spl predicted as spl:  0
num of others predicted as spl:  36916
Done: Traumatic-Surgery
Fitting 2 folds for each of 10 candidates, totalling 20 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best parameters found:  {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
X_train results,
X_test results,
number of spl predicted as others:  33
num of others predicted as spl:  1
num of spl predicted as spl:  2
num of others predicted as spl:  36914
[]
X_train results,
X_test results,
number of spl predicted as others:  33
num of others predicted as spl:  1
num of spl predicted as spl:  2
num of others predicted as spl:  36914
Done: Urgent Care-No Surgery
Fitting 2 folds for each of 10 candidates, totalling 20 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best parameters found:  {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
X_train results,
X_test results,
number of spl predicted as others:  57
num of others predicted as spl:  26
num of spl predicted as spl:  190
num of others predicted as spl:  36677
[]
X_train results,
X_test results,
number of spl predicted as others:  57
num of others predicted as spl:  26
num of spl predicted as spl:  190
num of others predicted as spl:  36677
Done: Urological-Surgery
